# Imports

In [1]:
import pandas as pd # import for dataframe handle
import numpy as np # import for math and array operations
import matplotlib.pyplot as plt # import for visual representation
#import seaborn as sns # import for visual representation

from bs4 import BeautifulSoup
import requests

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import string

# pipeline imports
from sklearn import set_config; set_config(display='diagram')
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import FunctionTransformer

# scalers, encoder, knn, vectorizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler

%load_ext autoreload
%autoreload 2

In [2]:
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
        
    lowercased = text.lower() # Lower Case
    lemmatizer = WordNetLemmatizer()
    #lemmatized = [lemmatizer.lemmatize(word) for word in lowercased]
    #lowercased = lemmatized     
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('English')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words]# Remove Stop Words
    lemmatized = [lemmatizer.lemmatize(word) for word in without_stopwords]
    
    return " ".join(lemmatized)

# Load Dataset 

In [3]:
df = pd.read_csv('../raw_data/clean_df.csv' )
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,achievements,op_sys
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","English,French,Italian,German,Spanish - Spain,...",Action,"About This Game Developed by id software, the...",0,17.991,Very Positive,2016-05-12 00:00:00,1.0,windows
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,1,26.991,Mixed,2017-12-21 00:00:00,1.0,windows
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","English,French,German,Russian","Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,0,35.991,Mostly Positive,2018-04-24 00:00:00,1.0,windows macOS Linux
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,0,40.491,Mixed,2018-12-13 00:00:00,0.0,windows
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,0,NaN,Mostly Positive,2003-05-06 00:00:00,0.0,windows macOS


In [4]:
df.isnull().sum()

url                     0
name                    3
developer              34
tags                    3
languages               0
genre                   5
game_description        0
mature_content          0
price                4558
reviews                 0
date                  259
achievements            0
op_sys              12383
dtype: int64

In [5]:
df['tags'].fillna('', inplace=True)
df['genre'].fillna('', inplace=True)

In [6]:
df['metadata'] = df[['tags', 'genre']].apply(lambda x: ' '.join(x), axis = 1)


df['metadata'] = df['metadata'].apply(
    lambda x: ' '.join(list(set(x.split())))
)

In [7]:
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,achievements,op_sys,metadata
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","English,French,Italian,German,Spanish - Spain,...",Action,"About This Game Developed by id software, the...",0,17.991,Very Positive,2016-05-12 00:00:00,1.0,windows,"Soundtrack,Multiplayer,Singleplayer,Fast-Paced..."
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,1,26.991,Mixed,2017-12-21 00:00:00,1.0,windows,"Multiplayer Shooter,Action,Online Person,Team-..."
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","English,French,German,Russian","Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,0,35.991,Mostly Positive,2018-04-24 00:00:00,1.0,windows macOS Linux,"Tactics,Sci-fi,Turn-Based Mechs,Strategy,Turn-..."
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,0,40.491,Mixed,2018-12-13 00:00:00,0.0,windows,"Access,Simulation,FPS,Post-apocalyptic,Surviva..."
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,0,NaN,Mostly Positive,2003-05-06 00:00:00,0.0,windows macOS,"Multiplayer,RPG,Strategy Play,Economy,Strategy..."


In [8]:
df['metadata'] = df['metadata'].apply(clean_text)

df['game_description'] = df['game_description'].apply(clean_text)

In [9]:
df['tags'] = df['tags'].apply(clean_text)

In [10]:
df['languages'] = df['languages'].apply(clean_text)

In [12]:
df['genre'] = df['genre'].apply(clean_text)

In [13]:
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,achievements,op_sys,metadata
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,fps gore action demon shooter first person gre...,english french italian german spanish spain ja...,action,game developed id software studio pioneered fi...,0,17.991,Very Positive,2016-05-12 00:00:00,1.0,windows,soundtrack multiplayer singleplayer fast paced...
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,survival shooter multiplayer battle royale pvp...,english korean simplified chinese french germa...,action adventure massively multiplayer,game playerunknown battleground battle royale ...,1,26.991,Mixed,2017-12-21 00:00:00,1.0,windows,multiplayer shooter action online person team ...
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,mechs strategy turn based turn based tactic sc...,english french german russian,action adventure strategy,game original battletech mechwarrior creator j...,0,35.991,Mostly Positive,2018-04-24 00:00:00,1.0,windows macOS Linux,tactic sci fi turn based mechs strategy turn b...
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,survival zombie open world multiplayer pvp mas...,english french italian german spanish spain cz...,action adventure massively multiplayer,game post soviet country chernarus struck unkn...,0,40.491,Mixed,2018-12-13 00:00:00,0.0,windows,access simulation fps post apocalyptic surviva...
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,space massively multiplayer sci fi sandbox mmo...,english german russian french,action free play massively multiplayer rpg str...,game,0,NaN,Mostly Positive,2003-05-06 00:00:00,0.0,windows macOS,multiplayer rpg strategy play economy strategy...


In [27]:
df.to_csv('../raw_data/clean_df.csv', index=False)

# Preprocessing

In [14]:
def create_pipeline(df):
    array_transf = FunctionTransformer(lambda array: array.toarray())
    
    meta_transf = make_pipeline(TfidfVectorizer(min_df=0.05), array_transf,RobustScaler())
    desc_transf = make_pipeline(TfidfVectorizer(min_df=0.1),array_transf, RobustScaler())
    
    ord_encoder = OrdinalEncoder(
        categories=[
            [
                "Overwhelmingly Negative",
                "Very Negative",
                "Negative",
                "Mostly Negative",
                'Mixed',
                "Mostly Positive",
                "Positive",
                "Very Positive",
                "Overwhelmingly Positive"
            ]],
        dtype=np.int64,
        handle_unknown="use_encoded_value",
        unknown_value=-1
    )
    
    ord_transf = make_pipeline(ord_encoder, StandardScaler())
    
    num_transf = make_pipeline(StandardScaler())


    preproc_basic = make_column_transformer(
        (meta_transf, 'metadata'),
        #(meta_transf, 'game_description'),
        (ord_transf, ['reviews']),
        #(num_transf, ['mature_content', 'achievements']),
        remainder='drop'
    )
    
    return preproc_basic.fit_transform(df)

In [15]:
def train(X, y):
    return KNeighborsRegressor().fit(X,y)

In [25]:
def recommending_system(model, X, game):
    
    neighbors_index = model.kneighbors(X.loc[[game]],n_neighbors=df.shape[0])[1][0]
    neighbors_distance = model.kneighbors(X.loc[[game]],n_neighbors=df.shape[0])[0][0]
    
    neighbors_list = list(neighbors_index)
    return pd.DataFrame(neighbors_distance, index = X.iloc[neighbors_list, :].index, columns=['distance']).head(10)
    

In [17]:
pipe = create_pipeline(df)
pipe.shape

(24567, 48)

In [18]:
X = pd.DataFrame(pipe, index=df.name.tolist())

In [19]:
model = train(X, df['url'])

In [26]:
recommending_system(model, X, 'Counter-Strike')

,distance
Counter-Strike,4.214685e-08
Quake Live™,7.401706e-01
Day of Defeat: Source,9.062712e-01
Darkest Hour: Europe '44-'45,9.290974e-01
Shadow Ops: Red Mercury,9.813751e-01
Heroes of The West,9.852454e-01
ShootMania Storm,9.953872e-01
Descenders,1.009885e+00
Counter-Strike: Global Offensive,1.013222e+00
Space Pirate Trainer,1.016252e+00
